In [1]:
import warnings  

import os
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset
from torch.nn.functional import softmax

from efficientnet_pytorch import EfficientNet

from typing import Callable, List, Tuple, Dict
from pathlib import Path

from transformers import AdamW
from transformers import get_cosine_schedule_with_warmup

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from collections import defaultdict, OrderedDict
from tqdm.notebook import tqdm
from torchsummary import summary

from fastai2.layers import LabelSmoothingCrossEntropy
from transformers import get_cosine_schedule_with_warmup
from transformers import AdamW

import matplotlib
matplotlib.rcParams.update({'figure.figsize': (16, 12), 'font.size': 14})
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

In [2]:
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", DeprecationWarning)
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
EXPERIMENT_NAME = "02_efficientnet-b0_tts=03"

class ConfigExperiment:
    logdir = f"./logs/{EXPERIMENT_NAME}"
    save_dirname = EXPERIMENT_NAME
    submission_file = f"{EXPERIMENT_NAME}.csv"
    seed = 42
    batch_size = 32
    model_name = 'efficientnet-b0'
    size = 512
    num_workers = 20
    root_images = "../../../data/raw/plant-pathology-2020-fgvc7/images/"
    root = "../../../data/raw/plant-pathology-2020-fgvc7/"
    num_classes = 4
    patience = 10
    early_stopping_delta = 1e-4
    num_epochs = 200
    lr = 0.003
    class_names = ["healthy", "multiple_diseases", "rust", "scab"]
    is_fp16_used = False
    
    
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    
config = ConfigExperiment()
set_seed(config.seed)
config.size = EfficientNet.get_image_size(config.model_name)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = "cuda" if torch.cuda.is_available() else "cpu"

try:
    # Create target Directory
    os.mkdir(config.save_dirname)
    print("Directory " , config.save_dirname ,  " Created ") 
except FileExistsError:
    print("Directory " , config.save_dirname ,  " already exists")

Directory  02_efficientnet-b0_tts=03  Created 


In [4]:
class PlantDataset(Dataset):
    
    def __init__(self, df, config, transforms=None):
    
        self.df = df
        self.images_dir = config.root_images
        self.class_names = config.class_names
        self.transforms=transforms
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image_src = self.images_dir + self.df.iloc[idx]['image_id'] + '.jpg'
        image = cv2.imread(image_src, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        labels = self.df.iloc[idx][self.class_names].values.astype(np.int8)
        label = torch.argmax(torch.from_numpy(labels))
        
        if self.transforms:
            transformed = self.transforms(image=image)
            image = transformed['image']

        return image, label

In [5]:
def pre_transforms(image_size=224):
    # Convert the image to a square of size image_size x image_size
    # (keeping aspect ratio)
    result = [
        A.LongestMaxSize(max_size=image_size),
        A.PadIfNeeded(image_size, image_size, border_mode=0)
    ]
    
    return result

def hard_transforms():
    result = [
        # Random shifts, stretches and turns with a 50% probability
#         A.RandomResizedCrop(height=config.size, width=config.size, p=1.0),
        A.OneOf([
            A.Rotate(limit=90, p=1),
            A.HorizontalFlip(p=1),
            A.VerticalFlip(p=1),
            A.OpticalDistortion(p=1),
            A.ShiftScaleRotate(rotate_limit=90, shift_limit=0.2, scale_limit=0.1, p=1)
        ], p=0.8),
        
        # Pixels
        A.OneOf([
            A.IAAEmboss(p=1.0),
            A.IAASharpen(p=1.0),
            A.Blur(p=1.0),
        ], p=0.5),

        # Affine
        A.OneOf([
            A.RandomContrast(limit=0.2, p=1),
            A.ElasticTransform(p=1),
            A.IAAPiecewiseAffine(p=1)
        ], p=0.5),
    ]
    
    return result

def post_transforms():
    # we use ImageNet image normalization
    # and convert it to torch.Tensor
    return [A.Normalize(p=1.0), ToTensorV2(p=1.0),]

def compose(transforms_to_compose):
    # combine all augmentations into one single pipeline
    result = A.Compose([item for sublist in transforms_to_compose for item in sublist])
    return result

In [6]:
train_df = pd.read_csv(config.root + 'train.csv')
train, valid = train_test_split(train_df, test_size=0.33, random_state=config.seed, shuffle=True, stratify=train_df[config.class_names])

train_transforms = compose([
    pre_transforms(config.size),
    hard_transforms(), 
    post_transforms()
])
valid_transforms = compose([
    pre_transforms(config.size), 
    post_transforms()
])

show_transforms = compose([
    pre_transforms(config.size),
    hard_transforms()
])

train_dataset = PlantDataset(train, config, train_transforms)
valid_dataset = PlantDataset(valid, config, valid_transforms)

train_dataloader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=config.num_workers)
valid_dataloader = DataLoader(valid_dataset, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)

In [7]:
def get_model(model_name: str, num_classes: int, pretrained: str = "imagenet") -> EfficientNet:
    model = EfficientNet.from_pretrained(model_name)
    for param in model.parameters():
        param.requires_grad = False
    num_ftrs = model._fc.in_features
    model._fc = nn.Sequential(nn.Linear(num_ftrs, num_classes, bias = True))
    return model

model = get_model(config.model_name, config.num_classes)

Loaded pretrained weights for efficientnet-b0


In [8]:
criterion = LabelSmoothingCrossEntropy()
optimizer = AdamW(model.parameters(), lr=config.lr, weight_decay=1e-3)

num_train_steps = int(len(train_dataset) / config.batch_size * config.num_epochs)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=len(train_dataset)/config.batch_size*5, num_training_steps=num_train_steps)

In [9]:
class Trainer:
    def __init__(self, model, train_dataloader: DataLoader, valid_dataloader: DataLoader, criterion, optimizer, scheduler, device, config: ConfigExperiment):
        self.model = model
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        self.config = config
        self.train_metrics = {
            'avg_loss': [],
            'auc/_mean': [],
            'auc/healthy': [],
            'auc/multiple_diseases': [],
            'auc/rust': [],
            'auc/scab': [],
        }
        self.valid_metrics = {
            'avg_loss': [],
            'auc/_mean': [],
            'auc/healthy': [],
            'auc/multiple_diseases': [],
            'auc/rust': [],
            'auc/scab': [],
        }
        self.counter = 0
        self.delta = config.early_stopping_delta
      
    def run(self):
        self.model.to(device)
        best_valid_loss = float('inf')
        best_valid_auc_mean = 0

        try:
            for i_epoch in tqdm(range(self.config.num_epochs), desc='Epochs', total=config.num_epochs, position=1, leave=True):
                start_time = time.time()

                train_loss, train_outputs, train_targets = self._train()
                valid_loss, valid_outputs, valid_targets = self._evaluate()
                    
                self.train_metrics["avg_loss"].append(train_loss)
                self.train_metrics["auc/_mean"].append(self.comp_metric(train_outputs, train_targets))
                self.train_metrics["auc/healthy"].append(self.healthy_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/multiple_diseases"].append(self.multiple_diseases_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/rust"].append(self.rust_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/scab"].append(self.scab_roc_auc(train_outputs, train_targets))
                
                self.valid_metrics["avg_loss"].append(valid_loss)
                self.valid_metrics["auc/_mean"].append(self.comp_metric(valid_outputs, valid_targets))
                self.valid_metrics["auc/healthy"].append(self.healthy_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/multiple_diseases"].append(self.multiple_diseases_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/rust"].append(self.rust_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/scab"].append(self.scab_roc_auc(valid_outputs, valid_targets))
                
                end_time = time.time()
                epoch_mins, epoch_secs = self._epoch_time(start_time, end_time)
                self.print_progress(i_epoch, epoch_mins, epoch_secs)
                
                self.scheduler.step(self.valid_metrics["auc/_mean"][-1])
                
                if valid_loss < best_valid_loss:
                    self.counter = 0
                    best_valid_loss = valid_loss
                    torch.save(model.state_dict(), f"{config.save_dirname}/best_model_epoch={i_epoch+1}_loss={best_valid_loss}.pth")
                else:
                    self.counter += 1
                    
                if self.valid_metrics["auc/_mean"][-1] > best_valid_auc_mean:
                    best_valid_auc_mean = self.valid_metrics["auc/_mean"][-1]
                    torch.save(model.state_dict(), f"{config.save_dirname}/best_model_epoch={i_epoch+1}_auc_mean={best_valid_auc_mean}.pth")
                    
                if self.counter > self.config.patience:
                    print("EarlyStopping")
                    break
        except KeyboardInterrupt: 
            pass
        
        return self.train_metrics, self.valid_metrics
        
    def _train(self):
        model.train()
        epoch_loss = 0
        epoch_output = None
        epoch_target = None
        for i, (images, labels) in tqdm(enumerate(self.train_dataloader), desc='Train', total=len(self.train_dataloader), position=2, leave=True):
            loss_iten, outputs = self._train_process(images, labels)
            epoch_loss += loss_iten              

            if epoch_output is None:
                epoch_output = outputs.cpu().data
            else:
                epoch_output = torch.cat((epoch_output, outputs.cpu().data))

            if epoch_target is None:
                epoch_target = labels.cpu().data
            else:
                epoch_target = torch.cat((epoch_target, labels.cpu().data))
            
        return epoch_loss / len(self.train_dataloader), epoch_output, epoch_target
    
    def _train_process(self, images, labels):
        images = images.to(self.device)
        labels = labels.to(self.device)
        self.optimizer.zero_grad()
        outputs = self.model(images)
        loss = self.criterion(outputs, labels)
        loss.backward()
        self.optimizer.step()
        return loss.item(), outputs
            
    def _evaluate(self):
        model.eval()
        epoch_loss = 0
        epoch_output = None
        epoch_target = None
        with torch.no_grad():
            for i, (images, labels) in enumerate(self.valid_dataloader):
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                epoch_loss += loss.item()
                
                if epoch_output is None:
                    epoch_output = outputs.cpu().data
                else:
                    epoch_output = torch.cat((epoch_output, outputs.cpu().data))

                if epoch_target is None:
                    epoch_target = labels.cpu().data
                else:
                    epoch_target = torch.cat((epoch_target, labels.cpu().data))

        return epoch_loss / len(self.valid_dataloader), epoch_output, epoch_target
    
#     def _evaluate_tta(self):
#         n_samples = len(dataset)
#         y_probas_tta = np.zeros((n_samples, config.num_classes, tta_count), dtype=np.float32)

#         model.eval()
#         model.to(device)
#         with torch.no_grad():
#             for i_epoch in tqdm(range(tta_count), desc='TTA', total=tta_count, position=1, leave=True):
#                 for i, (images, idx) in enumerate(dataloader):
#                     images = images.to(device)
#                     outputs = model(images)
#                     y_pred = F.softmax(outputs, dim=1)

#                     tta_index = i_epoch - 1
#                     start_index = (i % len(dataloader)) * config.batch_size
#                     end_index = min(start_index + config.batch_size, n_samples)
#                     batch_y_probas = y_pred.detach().cpu().numpy()                
#                     y_probas_tta[start_index:end_index, :, tta_index] = batch_y_probas

#         return y_probas_tta

#     def show_history(self):
#         fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 8))
#         clear_output(True)
#         ax[0].plot(history, label='train loss')
#         ax[0].set_xlabel('Batch')
#         ax[0].set_title('Train loss')
#         if train_history is not None:
#             ax[1].plot(train_history, label='general train history')
#             ax[1].set_xlabel('Epoch')
#         if valid_history is not None:
#             ax[1].plot(valid_history, label='general valid history')
#         plt.legend()
#         plt.show()

    
    def _epoch_time(self, start_time, end_time):
        elapsed_time = end_time - start_time
        elapsed_mins = int(elapsed_time / 60)
        elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
        return elapsed_mins, elapsed_secs

    def print_progress(self, i_epoch, epoch_mins, epoch_secs):
        i_epoch = i_epoch + 1
        print(f"Epoch: {i_epoch:02} | Time: {epoch_mins}m {epoch_secs}s")
        print("Training Results - Average Loss: {:.4f} | auc/_mean: {:.4f} | auc/healthy: {:.4f} | auc/multiple_diseases: {:.4f}, auc/rust: {:.4f}, auc/scab: {:.4f}"
            .format(
                self.train_metrics['avg_loss'][-1], 
                self.train_metrics['auc/_mean'][-1],
                self.train_metrics['auc/healthy'][-1],
                self.train_metrics['auc/multiple_diseases'][-1],
                self.train_metrics['auc/rust'][-1],
                self.train_metrics['auc/scab'][-1],
            ))
        print("Evaluating Results - Average Loss: {:.4f} | auc/_mean: {:.4f} | auc/healthy: {:.4f} | auc/multiple_diseases: {:.4f}, auc/rust: {:.4f}, auc/scab: {:.4f}"
            .format( 
                self.valid_metrics['avg_loss'][-1], 
                self.valid_metrics['auc/_mean'][-1],
                self.valid_metrics['auc/healthy'][-1],
                self.valid_metrics['auc/multiple_diseases'][-1],
                self.valid_metrics['auc/rust'][-1],
                self.valid_metrics['auc/scab'][-1],
            ))
        print()
        
    def comp_metric(self, preds, targs, labels=range(4)):
        preds = torch.sigmoid(preds)
        targs = torch.eye(4)[targs]
        return np.mean([roc_auc_score(targs[:,i], preds[:,i]) for i in labels])

    def healthy_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[0])

    def multiple_diseases_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[1])

    def rust_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[2])

    def scab_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[3])



In [10]:
trainer = Trainer(model, train_dataloader, valid_dataloader, criterion, optimizer, scheduler, device, config)
trainer.run();


Epoch: 01 | Time: 0m 14s
Training Results - Average Loss: 1.3986 | auc/_mean: 0.4870 | auc/healthy: 0.4904 | auc/multiple_diseases: 0.4943, auc/rust: 0.4459, auc/scab: 0.5175
Evaluating Results - Average Loss: 1.3815 | auc/_mean: 0.4503 | auc/healthy: 0.5165 | auc/multiple_diseases: 0.5409, auc/rust: 0.2805, auc/scab: 0.4632




Epoch: 02 | Time: 0m 14s
Training Results - Average Loss: 1.4057 | auc/_mean: 0.4540 | auc/healthy: 0.4529 | auc/multiple_diseases: 0.4375, auc/rust: 0.4449, auc/scab: 0.4807
Evaluating Results - Average Loss: 1.3745 | auc/_mean: 0.4570 | auc/healthy: 0.4774 | auc/multiple_diseases: 0.5663, auc/rust: 0.3099, auc/scab: 0.4745




Epoch: 03 | Time: 0m 13s
Training Results - Average Loss: 1.3910 | auc/_mean: 0.4766 | auc/healthy: 0.4746 | auc/multiple_diseases: 0.4855, auc/rust: 0.4557, auc/scab: 0.4907
Evaluating Results - Average Loss: 1.3701 | auc/_mean: 0.4620 | auc/healthy: 0.4311 | auc/multiple_diseases: 0.5693, auc/rust: 0.3541, auc/scab: 0.4934




Epoch: 04 | Time: 0m 13s
Training Results - Average Loss: 1.3809 | auc/_mean: 0.4711 | auc/healthy: 0.4771 | auc/multiple_diseases: 0.4634, auc/rust: 0.4586, auc/scab: 0.4855
Evaluating Results - Average Loss: 1.3613 | auc/_mean: 0.4703 | auc/healthy: 0.4172 | auc/multiple_diseases: 0.5434, auc/rust: 0.3898, auc/scab: 0.5310




Epoch: 05 | Time: 0m 13s
Training Results - Average Loss: 1.3753 | auc/_mean: 0.4858 | auc/healthy: 0.4688 | auc/multiple_diseases: 0.4729, auc/rust: 0.4670, auc/scab: 0.5343
Evaluating Results - Average Loss: 1.3499 | auc/_mean: 0.4829 | auc/healthy: 0.4251 | auc/multiple_diseases: 0.5213, auc/rust: 0.4199, auc/scab: 0.5652



...


Epoch: 196 | Time: 0m 13s
Training Results - Average Loss: 0.9892 | auc/_mean: 0.8076 | auc/healthy: 0.8569 | auc/multiple_diseases: 0.6220, auc/rust: 0.8779, auc/scab: 0.8735
Evaluating Results - Average Loss: 0.8536 | auc/_mean: 0.8375 | auc/healthy: 0.9242 | auc/multiple_diseases: 0.5368, auc/rust: 0.9530, auc/scab: 0.9360




Epoch: 197 | Time: 0m 13s
Training Results - Average Loss: 0.9941 | auc/_mean: 0.7898 | auc/healthy: 0.8288 | auc/multiple_diseases: 0.5749, auc/rust: 0.8819, auc/scab: 0.8736
Evaluating Results - Average Loss: 0.8530 | auc/_mean: 0.8390 | auc/healthy: 0.9245 | auc/multiple_diseases: 0.5427, auc/rust: 0.9533, auc/scab: 0.9355




Epoch: 198 | Time: 0m 13s
Training Results - Average Loss: 1.0052 | auc/_mean: 0.8044 | auc/healthy: 0.8418 | auc/multiple_diseases: 0.6274, auc/rust: 0.8823, auc/scab: 0.8663
Evaluating Results - Average Loss: 0.8531 | auc/_mean: 0.8394 | auc/healthy: 0.9241 | auc/multiple_diseases: 0.5448, auc/rust: 0.9534, auc/scab: 0.9353




Epoch: 199 | Time: 0m 13s
Training Results - Average Loss: 1.0235 | auc/_mean: 0.7819 | auc/healthy: 0.8333 | auc/multiple_diseases: 0.5886, auc/rust: 0.8670, auc/scab: 0.8387
Evaluating Results - Average Loss: 0.8510 | auc/_mean: 0.8405 | auc/healthy: 0.9253 | auc/multiple_diseases: 0.5469, auc/rust: 0.9540, auc/scab: 0.9357




Epoch: 200 | Time: 0m 13s
Training Results - Average Loss: 1.0121 | auc/_mean: 0.7891 | auc/healthy: 0.8210 | auc/multiple_diseases: 0.6153, auc/rust: 0.8695, auc/scab: 0.8506
Evaluating Results - Average Loss: 0.8486 | auc/_mean: 0.8405 | auc/healthy: 0.9261 | auc/multiple_diseases: 0.5451, auc/rust: 0.9548, auc/scab: 0.9360




In [11]:
model = get_model(config.model_name, config.num_classes)
model.load_state_dict(torch.load(f"{config.save_dirname}/best_model_epoch=200_auc_mean=0.8405395713741826.pth"))
for param in model.parameters():
    param.requires_grad = True

criterion = LabelSmoothingCrossEntropy()
optimizer = AdamW(model.parameters(), lr=config.lr, weight_decay=1e-3)
num_train_steps = int(len(train_dataset) / config.batch_size * config.num_epochs)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=len(train_dataset)/config.batch_size*5, num_training_steps=num_train_steps)

Loaded pretrained weights for efficientnet-b0


In [12]:
trainer = Trainer(model, train_dataloader, valid_dataloader, criterion, optimizer, scheduler, device, config)
trainer.run();


Epoch: 01 | Time: 0m 19s
Training Results - Average Loss: 1.0017 | auc/_mean: 0.8102 | auc/healthy: 0.8540 | auc/multiple_diseases: 0.6370, auc/rust: 0.8753, auc/scab: 0.8744
Evaluating Results - Average Loss: 0.8471 | auc/_mean: 0.8399 | auc/healthy: 0.9266 | auc/multiple_diseases: 0.5413, auc/rust: 0.9549, auc/scab: 0.9367




Epoch: 02 | Time: 0m 18s
Training Results - Average Loss: 0.9765 | auc/_mean: 0.8032 | auc/healthy: 0.8493 | auc/multiple_diseases: 0.5903, auc/rust: 0.8911, auc/scab: 0.8822
Evaluating Results - Average Loss: 0.8000 | auc/_mean: 0.8477 | auc/healthy: 0.9407 | auc/multiple_diseases: 0.5385, auc/rust: 0.9652, auc/scab: 0.9464




Epoch: 03 | Time: 0m 19s
Training Results - Average Loss: 0.9393 | auc/_mean: 0.8164 | auc/healthy: 0.8772 | auc/multiple_diseases: 0.5869, auc/rust: 0.9081, auc/scab: 0.8935
Evaluating Results - Average Loss: 0.7627 | auc/_mean: 0.8553 | auc/healthy: 0.9518 | auc/multiple_diseases: 0.5438, auc/rust: 0.9720, auc/scab: 0.9535




Epoch: 04 | Time: 0m 19s
Training Results - Average Loss: 0.9189 | auc/_mean: 0.8319 | auc/healthy: 0.8687 | auc/multiple_diseases: 0.6343, auc/rust: 0.9249, auc/scab: 0.8997
Evaluating Results - Average Loss: 0.7318 | auc/_mean: 0.8626 | auc/healthy: 0.9593 | auc/multiple_diseases: 0.5553, auc/rust: 0.9767, auc/scab: 0.9590




Epoch: 05 | Time: 0m 18s
Training Results - Average Loss: 0.8695 | auc/_mean: 0.8544 | auc/healthy: 0.8968 | auc/multiple_diseases: 0.6650, auc/rust: 0.9383, auc/scab: 0.9176
Evaluating Results - Average Loss: 0.7060 | auc/_mean: 0.8670 | auc/healthy: 0.9653 | auc/multiple_diseases: 0.5592, auc/rust: 0.9802, auc/scab: 0.9634



...


Epoch: 140 | Time: 0m 18s
Training Results - Average Loss: 0.4392 | auc/_mean: 0.9902 | auc/healthy: 0.9975 | auc/multiple_diseases: 0.9712, auc/rust: 0.9943, auc/scab: 0.9979
Evaluating Results - Average Loss: 0.4702 | auc/_mean: 0.9746 | auc/healthy: 0.9976 | auc/multiple_diseases: 0.9085, auc/rust: 0.9978, auc/scab: 0.9944




Epoch: 141 | Time: 0m 18s
Training Results - Average Loss: 0.4471 | auc/_mean: 0.9912 | auc/healthy: 0.9984 | auc/multiple_diseases: 0.9714, auc/rust: 0.9963, auc/scab: 0.9985
Evaluating Results - Average Loss: 0.4680 | auc/_mean: 0.9745 | auc/healthy: 0.9976 | auc/multiple_diseases: 0.9078, auc/rust: 0.9978, auc/scab: 0.9948




Epoch: 142 | Time: 0m 19s
Training Results - Average Loss: 0.4397 | auc/_mean: 0.9904 | auc/healthy: 0.9981 | auc/multiple_diseases: 0.9695, auc/rust: 0.9973, auc/scab: 0.9967
Evaluating Results - Average Loss: 0.4682 | auc/_mean: 0.9740 | auc/healthy: 0.9976 | auc/multiple_diseases: 0.9058, auc/rust: 0.9978, auc/scab: 0.9946




Epoch: 143 | Time: 0m 19s
Training Results - Average Loss: 0.4319 | auc/_mean: 0.9961 | auc/healthy: 0.9980 | auc/multiple_diseases: 0.9919, auc/rust: 0.9985, auc/scab: 0.9959
Evaluating Results - Average Loss: 0.4693 | auc/_mean: 0.9750 | auc/healthy: 0.9976 | auc/multiple_diseases: 0.9101, auc/rust: 0.9979, auc/scab: 0.9944




Epoch: 144 | Time: 0m 18s
Training Results - Average Loss: 0.4381 | auc/_mean: 0.9939 | auc/healthy: 0.9964 | auc/multiple_diseases: 0.9835, auc/rust: 0.9984, auc/scab: 0.9974
Evaluating Results - Average Loss: 0.4681 | auc/_mean: 0.9752 | auc/healthy: 0.9976 | auc/multiple_diseases: 0.9109, auc/rust: 0.9979, auc/scab: 0.9942

EarlyStopping


In [13]:

class PlantDatasetTest(Dataset):
    
    def __init__(self, df, config, transforms=None):
    
        self.df = df
        self.images_dir = config.root_images
        self.class_names = config.class_names
        self.transforms=transforms
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image_src = self.images_dir + self.df.iloc[idx]['image_id'] + '.jpg'
        image = cv2.imread(image_src, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms:
            transformed = self.transforms(image=image)
            image = transformed['image']

        return image, idx

test_df = pd.read_csv(config.root + 'test.csv')
test_dataset = PlantDatasetTest(test_df, config, train_transforms)
test_dataloader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)

In [14]:
def predict(model, dataloader: DataLoader, device, dataset: Dataset, tta_count: int, config: ConfigExperiment):
    n_samples = len(dataset)
    y_probas_tta = np.zeros((n_samples, config.num_classes, tta_count), dtype=np.float32)
    
    model.eval()
    model.to(device)
    with torch.no_grad():
        for i_epoch in tqdm(range(tta_count), desc='TTA', total=tta_count, position=1, leave=True):
            for i, (images, idx) in enumerate(dataloader):
                images = images.to(device)
                outputs = model(images)
                y_pred = F.softmax(outputs, dim=1)
                
                tta_index = i_epoch - 1
                start_index = (i % len(dataloader)) * config.batch_size
                end_index = min(start_index + config.batch_size, n_samples)
                batch_y_probas = y_pred.detach().cpu().numpy()                
                y_probas_tta[start_index:end_index, :, tta_index] = batch_y_probas

    return y_probas_tta

In [15]:
model = get_model(config.model_name, config.num_classes)
model.load_state_dict(torch.load(f"{config.save_dirname}/best_model_epoch=126_auc_mean=0.9753065692644196.pth"))

Loaded pretrained weights for efficientnet-b0


<All keys matched successfully>

In [16]:
result = predict(model, test_dataloader, device, test_dataset, tta_count=5, config=config)

In [17]:
y_probas = np.mean(result, axis=-1)

In [18]:
y_probas[:10]

array([[0.04094496, 0.09230677, 0.85037744, 0.01637086],
       [0.01575707, 0.04604911, 0.92792654, 0.01026727],
       [0.01152406, 0.02910677, 0.01681365, 0.94255555],
       [0.95874006, 0.0132782 , 0.01732136, 0.01066045],
       [0.01898009, 0.01973237, 0.9515273 , 0.00976023],
       [0.81728727, 0.01715877, 0.08577295, 0.07978107],
       [0.9641827 , 0.00806555, 0.02274434, 0.00500738],
       [0.01878336, 0.20291618, 0.04592507, 0.73237544],
       [0.02047195, 0.4191211 , 0.12451655, 0.43589044],
       [0.05369949, 0.07008991, 0.83317363, 0.04303695]], dtype=float32)

In [19]:
test_df = pd.read_csv(config.root + 'test.csv')
test_df["healthy"] = 0
test_df["multiple_diseases"] = 0
test_df["rust"] = 0
test_df["scab"] = 0
test_df[['healthy', 'multiple_diseases', 'rust', 'scab']] = y_probas
test_df.to_csv(config.submission_file, index=False)
test_df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.040945,0.092307,0.850377,0.016371
1,Test_1,0.015757,0.046049,0.927927,0.010267
2,Test_2,0.011524,0.029107,0.016814,0.942556
3,Test_3,0.958740,0.013278,0.017321,0.010660
4,Test_4,0.018980,0.019732,0.951527,0.009760


In [20]:
config.submission_file

'02_efficientnet-b0_tts=03.csv'